<a href="https://colab.research.google.com/github/mingd00/News-RAG/blob/main/news_api2json_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
from urllib.parse import urlparse
from email.utils import parsedate_to_datetime

In [2]:
def get_article_content(url: str) -> str:
    """
    주어진 URL에서 기사 본문을 추출하는 함수.

    :param url: 뉴스 기사 페이지의 URL
    :return: 기사 본문 텍스트 (사진은 제외됨)
    """
    try:
        # 웹 페이지 요청 보내기
        response = requests.get(url)

        # 요청이 성공적일 때
        if response.status_code == 200:
            # BeautifulSoup을 사용해 HTML 파싱
            soup = BeautifulSoup(response.text, 'html.parser')

            # 기사 본문이 들어있는 <article> 태그 찾기
            article = soup.find('article', {'id': 'dic_area', 'class': 'go_trans _article_content'})

            # 기사 본문 추출
            if article:
                content = article.get_text(strip=True)
                return content
            else:
                return "기사 본문을 찾을 수 없습니다."
        else:
            return f"웹사이트 요청 실패: {response.status_code}"

    except Exception as e:
        return f"오류 발생: {str(e)}"


# 사용 예시
url = 'https://n.news.naver.com/mnews/article/421/0008233058?sid=102'  # 여기에 실제 뉴스 기사 URL을 넣으세요
article_content = get_article_content(url)
print(article_content)

최경주가 2024년 5월15일 제주 서귀포 핀크스 콜프클럽에서 열린 한국프로골프투어(KPGA) SK텔레콤 채리티 오픈(자선경기)에서 10번홀 세컨 아이언샷을 하고 있다. (SK텔레콤 제공) 2024.5.15/뉴스1(제주=뉴스1) 고동명 기자 = 5월 한 달간 제주도내에서 총 22개 스포츠대회가 개최된다고 6일 제주도가 밝혔다.도에 따르면 국제대회로는 오는 24~25일 탑동공원과 성판악 일대에서 '제24회 제주 국제 울트라마라톤 대회'가 열린다. 이 대회에선 10개국 선수와 관계자 등 200여 명이 참가한다.전국대회로는 8일 'KPGA 클래식 골프대회'와 15일 'SK텔레콤2025 골프대회'가 연달아 개최된다. 전국에서 모인 프로골프선수 240여 명이 이들 대회에서 최강자를 가린다. 15일에는 '제80회 '제주 전국 학생테니스 선수권대회'도 예정돼 있다.도내 대회는 지난 3일 열린 제3회 '제주특별자치도체육회장배 전도파크골프대회'를 시작으로 4일 '2025 도족구협회장배 전도족구대회', 10일 '2025 제37회 미스터제주 챔피언십', 17일 '제17회 미디어제주기 전도 유소년 축구대회' 등이 도내 곳곳에서 펼쳐진다.이외에도 △2025 제주 챌린저컵 유소년 축구 리그 대회(3~4일, 외도구장) △제26회 KCTV배 제주도 배드민턴 대회(10~11일, 제주복합체육관) △제25회 도협회장배 전도여성게이트볼대회(24일, 제주복합체육관) 등 다양한 종목 대회가 열린다.장애인대회로는 9일 '제19회 제주도지사배 전국장애인사이클대회'를 시작으로 11일 '제3회 제주도장애인 수영연맹회장배 어울림수영대회', 15일 '제16회 전도시각장애인플라잉디스크대회 및 한·일 국제플라잉디스크교류대회', 31일 '제4회 제주시 어울림 생활체육대회'가 열린다.도는 이달 열리는 국제대회에선 11억 2600만 원, 전국대회는 80억 8000만 원, 도내대회는 3억 8200만 원 등의 지역경제 파급효과를 예상하고 있다.


In [3]:
news_media_mapping = {
    "yonhapnews.co.kr": "연합뉴스",
    "yonhapnewstv.co.kr": "연합뉴스TV",
    "news1.kr": "뉴스1",
    "edu.donga.com": "동아일보",
    "biz.heraldcorp.com": "헤럴드경제",
    "daily.hankooki.com": "한국일보",
    "kmib.co.kr": "기독교일보",
    "kbs.co.kr": "KBS",
    "munhwa.com": "문화일보",
    "sports.naver.com": "네이버 스포츠",
    "newsis.com": "뉴시스",
    "segye.com": "세계일보",
    "hankooki.com": "한겨레",
    "chosun.com": "조선일보",
    "joongang.co.kr": "중앙일보",
    "khan.co.kr": "경향신문",
    "hani.co.kr": "한겨레",
    "sports.chosun.com": "스포츠조선",
    "sports.donga.com": "스포츠동아",
    "news.naver.com": "네이버 뉴스",
    "n.news.naver.com": "네이버 뉴스",
    "m.post.naver.com": "네이버 포스트",
    "seoul.co.kr": "서울신문",
    "etnews.com": "전자신문",
    "ytn.co.kr": "YTN",
    "tbs.seoul.kr": "TBS",
    "mbn.co.kr": "MBN 뉴스",
    "news.kmib.co.kr": "국민일보",
    "newdaily.co.kr": "뉴데일리",
    "yna.co.kr": "연합뉴스",
    "naeil.com": "내일신문",
    "kihoilbo.co.kr": "기호일보",
    "edaily.co.kr": "이데일리",
    "fnnews.com": "파이넨셜뉴스",
    "hankyung.com": "한경",
    "incheonnews.com": "인천뉴스",
    "bloter.net": "BROTER",
    "dt.co.kr": "디지털타임스",
    "sentv.co.kr": "서울경제TV" ,
    "econovill.com": "이코노믹 리뷰",
    "nytimes.com": "The New York Times",
    "digitaltoday.co.kr": "Digital Today",
    "koreajoongangdaily.joins.com": "Korea JoongAng Daily",
    "ddaily.co.kr": "디지털 데일리",
    "hankyung.com": "한국경제신문",
    "sedaily.com": "서울경제",
    "sportsseoul.com": "스포츠서울",
    "joongboo.com": "중부일보",
    "nocutnews.co.kr": "노컷뉴스",
    "imbc.com": "MBC 뉴스",
    "news.mt.co.kr": "머니투데이",
}

In [4]:
# URL에서 도메인 추출 함수
def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.hostname
    if domain.startswith("www."):
        domain = domain[4:]  # "www." 제거
    return domain

In [5]:
# 태그 제거
def clean_html_tags(text):
    return re.sub(r"<.*?>", "", text)

# 날짜 형식 지정
def format_date(pub_date_str):
    try:
        dt = parsedate_to_datetime(pub_date_str)
        return dt.strftime("%Y-%m-%d %H:%M")  # 예: 2025-05-06 10:59
    except Exception as e:
        print(f"날짜 파싱 오류: {e}")
        return pub_date_str

In [6]:
# 네이버 API 키 (자신의 API 키로 교체하세요)
client_id = "gh33NVdUbiTCVrbFAQhV"
client_secret = "Wg6SoJlOj6"

# 뉴스 데이터를 담을 리스트
news_data = []

In [7]:
# 요청 URL을 위한 기본 설정
base_url = "https://openapi.naver.com/v1/search/news.json"

# 네이버 API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

In [8]:
keywords = [
    "정치", "경제", "사회", "문화", "IT", "스포츠"
]

In [9]:
total_news_count = 500
display_count = 100
total_pages = total_news_count // display_count

news_data = []

for keyword in keywords:
    print(f"🔍 '{keyword}' 키워드 수집 시작, 현재까지 리스트 길이: {len(news_data)}")

    for start in range(1, total_pages + 1):
        params = {
            "query": keyword,
            "display": display_count,
            "start": (start - 1) * display_count + 1,
            "sort": "sim"
        }

        try:
            response = requests.get(base_url, headers=headers, params=params)
            data = response.json()

            for item in data["items"]:
                if item["link"].startswith("https://n.news.naver.com/"):
                    news_item = {
                        "title": clean_html_tags(item["title"]),
                        "date": format_date(item["pubDate"]),
                        "link": item["link"],
                        "content": get_article_content(item["link"]),
                        "journal": news_media_mapping.get(extract_domain(item["originallink"]), "Unknown")
                    }
                    news_data.append(news_item)

            time.sleep(1)

        except Exception as e:
            print(f"❌ '{keyword}' 키워드의 {start}페이지에서 오류 발생: {e}")
            break

print(f"✅ 전체 수집 완료! 총 수집된 기사 수: {len(news_data)}")


🔍 '정치' 키워드 수집 시작, 현재까지 리스트 길이: 0
🔍 '경제' 키워드 수집 시작, 현재까지 리스트 길이: 408
🔍 '사회' 키워드 수집 시작, 현재까지 리스트 길이: 713
🔍 '문화' 키워드 수집 시작, 현재까지 리스트 길이: 935
🔍 'IT' 키워드 수집 시작, 현재까지 리스트 길이: 1161
❌ 'IT' 키워드의 5페이지에서 오류 발생: 'NoneType' object has no attribute 'startswith'
🔍 '스포츠' 키워드 수집 시작, 현재까지 리스트 길이: 1353
✅ 전체 수집 완료! 총 수집된 기사 수: 1519


In [11]:
# 결과를 JSON 파일로 저장
with open("news.json", "w", encoding="utf-8") as f:
    json.dump(news_data, f, ensure_ascii=False, indent=4)

print(f"총 {len(news_data)}개의 뉴스가 저장되었습니다.")

총 1519개의 뉴스가 저장되었습니다.


In [12]:
# "journal" 값이 "Unknown"인 데이터 개수 세기
unknown_count = sum(1 for item in news_data if item['journal'] == 'Unknown')

# 개수 출력
print(f'Unknown인 데이터 개수: {unknown_count}')

Unknown인 데이터 개수: 546


In [13]:
# "journal" 값이 "Unknown"이 아닌 데이터만 남김
filtered_data = [item for item in news_data if item['journal'] != 'Unknown']

# 새로운 JSON 파일로 저장
with open('news_data.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f'Unknown 값을 제거한 후 남은 데이터 개수: {len(filtered_data)}')

Unknown 값을 제거한 후 남은 데이터 개수: 973


In [14]:
# 저장된 JSON 파일 열기
with open("news_data.json", "r", encoding="utf-8") as f:
    news_data = json.load(f)

# 첫 번째 뉴스의 content 확인
first_news_content = news_data[0]

print(first_news_content)

{'title': '국힘 &quot;이재명 면죄입법 철회해야…정치압력에 李공판 연기&quot;', 'date': '2025-05-07 15:52', 'link': 'https://n.news.naver.com/mnews/article/001/0015373742?sid=100', 'content': '의총서 민주당 선거법·형소법 개정안 추진 규탄권성동 "李공판기일 연기 참으로 유감·개탄스럽다"(서울=연합뉴스) 류미나 조다운 기자 = 국민의힘은 7일 더불어민주당이 형사소송법 및 공직선거법 개정안을 추진하는 데 대해 "피고인 이재명 면죄입법을 즉시 철회하라"고 촉구했다.국민의힘은 이날 오후 의원총회에서 대통령 당선 시 진행 중인 형사 재판을 정지시키는 내용의 형사소송법 개정안과 허위사실공표죄 조항을 손질하는 공직선거법 개정안 처리에 나선 민주당을 향해 이같이 요구했다.권성동 원내대표는 모두발언에서 "법제사법위원회와 행정안전위원회에서는 일제히 이재명의 유죄를 무산시키는 법안을 상정했다"고 말했다.이어 "이재명을 처벌 못 하도록 허위사실공표죄를 개정하는 선거법 개정안, 이재명이 대통령이 되면 재판을 중단시키는 형소법 개정안을 강행 처리할 예정"이라며 "이런 무도한 집단이 깡패 집단이지 정당이라고 할 수 있나"라고 비난했다.그러면서 "민주당에 제안한다. 차라리 \'이재명 유죄 금지법\'을 제정하라. 이 사람은 신성불가침의 존재이니까 무조건 무죄라고 쓰고 일방 처리하라"고 꼬집었다.권 원내대표는 "왜 애꿎은 허위사실공표죄를 개정하는가"라며 "위증교사죄를 폐지하고, \'대장동·백현동 배임죄\'를 폐지하라. \'김정은 통치자금 상납법\', \'유엔대북제재 탈퇴법\'은 왜 제정 안 하나"라고 비꼬았다.아울러 "국회는 범죄자 이재명을 위한 면죄부 발급의 도구로 전락했다"며 "이재명을 히틀러, 스탈린에게 비유하기도 아깝다. 유죄 판결을 내렸다고 대법원장, 대법관을 고발하는 이재명 세력의 행태를 보면 \'표현의 자유\'는 있지만 \'표현 이후의 자유\'는 보장해줄 수 없다고 했던 아

In [15]:
# 조건에 맞는 항목 개수 세기
count = sum(1 for item in news_data if item.get("content") == "본문을 크롤링할 수 없습니다.")

print(f'"본문을 크롤링할 수 없습니다."인 항목 수: {count}')

"본문을 크롤링할 수 없습니다."인 항목 수: 0
